5.3.1 Text Classification
• Design and implement a system to recommend a conference to a researcher given the title of his new article.
• The system should use the provided Conference Proceedings training data. You should implement the sub-tasks (feature extraction, dimensionality reduction, and classifier) by yourself.
• You are free to select the algorithms you prefer for each sub-task. However, it is recommended that you test and compare multiple methods.
• Evaluate you system on the training set by using the cross-validation approach. Provide the confusion
matrix of your system output.
• Evaluation should be done in terms of Micro-average precision, recall and F1 measures.
• Once you found the best model on the training set, evaluate your model on the test set and report the
results.

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
train_data = pd.read_csv('DBLPTrainset.txt', sep='\t', header=None, names=['ID', 'Conference', 'Title'])
test_data = pd.read_csv('DBLPTestset.txt', sep='\t', header=None, names=['ID', 'Title'])
ground_truth_labels = pd.read_csv('DBLPTestGroundTruth.txt', sep='\t', header=None, names=['ID', 'Conference'])

In [21]:
# TF-IDF Vectorization for both training and test sets
tfidf_Vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_Vectorizer.fit_transform(train_data['Title'])
y_train = train_data['Conference']

X_test_tfidf = tfidf_Vectorizer.transform(test_data['Title'])

In [19]:
# Feature Extraction - Word Embeddings (Word2Vec)
word2vec_model = Word2Vec(sentences=train_data['Title'].apply(str.split), vector_size=100, window=5, min_count=1, workers=4)

def get_word_vector(words):
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(word2vec_model.vector_size)

X_train_word2vec = pd.DataFrame([get_word_vector(words) for words in train_data['Title'].apply(str.split)])
X_test_word2vec = pd.DataFrame([get_word_vector(words) for words in test_data['Title'].apply(str.split)])

In [26]:
# Initialize an empty DataFrame to store results
results = []

In [ ]:
# SVM with TF-IDF
svm_tfidf_classifier = SVC(kernel='linear', C=1.0)
svm_tfidf_classifier.fit(X_train_tfidf, train_data['Conference'])
y_pred_test_svm_tfidf = svm_tfidf_classifier.predict(X_test_tfidf)
accuracy_svm_tfidf = accuracy_score(ground_truth_labels['Conference'], y_pred_test_svm_tfidf)
results.append({'Features': 'TF-IDF', 'Classifier': 'SVM', 'Accuracy': accuracy_svm_tfidf})

In [7]:
# Combine test data with predicted labels and ground truth labels
evaluation_data = pd.DataFrame({'ID': test_data['ID'], 'Predicted_Conference_Label': y_pred_test})
evaluation_data = pd.merge(evaluation_data, ground_truth_labels, on='ID')

In [8]:
# Calculate accuracy
accuracy = (evaluation_data['Predicted_Conference_Label'] == evaluation_data['Conference']).mean()
print(f"Accuracy on the test set: {accuracy}")

Accuracy on the test set: 0.8639193596205159


5.3.2 Named Entity Recognition
• Design and implement a system to extract and classify named entities in tweets.
• The system should use the provided NER Twitter training data. You should implement the sub-tasks (feature extraction, and classifier) by yourself.
• You are free to select the algorithms you prefer for each sub-task. However, it is recommended that you test and compare multiple methods.
• Evaluate you system on the training set by using the cross-validation approach.
• Evaluation should be done in terms of micro-average of precision, recall and F1 measures.
• Evaluate your best model you found on the training set on the test set. Report your results.